<a href="https://colab.research.google.com/github/MunJinSeo/MachineLearning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5(AAI107)_Team13_%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_20220608(%ED%9A%8C%EC%82%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 팍스넷, 네이버 증권 뉴스 크롤링
# https://github.com/CSJ516/Stock_Prediction_project

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from datetime import datetime

In [ ]:
# all_data 대상일자 추출
#target_date =  all_data.index.strftime('%Y%m%d').copy() # pd.DataFrame(all_data.index).copy()
target_date = {"Date":['20220601','20220602','20220603']}
df_target_date = pd.DataFrame(target_date)
print(df_target_date)
print(type(df_target_date))
print(df_target_date["Date"].tail(1))

       Date
0  20220601
1  20220602
2  20220603
<class 'pandas.core.frame.DataFrame'>
2    20220603
Name: Date, dtype: object


In [ ]:
def paxnet_news_title(date_list):
  # 팍스넷 > 뉴스 > 많이본뉴스 > 증권
  paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
  #paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&objId=N4667'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  paxnet_result = []
  paxnet_error_cnt = 0
  
  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = paxnet_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.thumb-list li')
        for ttt in title_list:
          #print(ttt)
          try:
            news_title = ttt.select_one('dl.text > dt').text.strip()
            paxnet_result.append([news_title])
          except:
            paxnet_error_cnt += 1
  return paxnet_result

In [ ]:
paxnet_news_rs = paxnet_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
#paxnet_news_rs = paxnet_news_title(df_target_date["Date"].head(1)) # 샘플로 최초 1일자만 추출해봄
print("paxnet_news_rs length == ", len( paxnet_news_rs ) )

http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo=1&genDate=20220603&objId=N4667
paxnet_news_rs length ==  20


In [ ]:
paxnet_title_df = pd.DataFrame(paxnet_news_rs, columns=['뉴스제목'])
#paxnet_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
paxnet_title_df

,뉴스제목
0,국내 환율 마감시황
1,FTSE100(영국) : Platinum Jubilee Bank Holiday로 휴장
2,국내 채권금리 동향(마감)
3,항셍지수(홍콩) : 단오절(Tuen Ng Festival)로 휴장
4,뉴욕증시 마감시황
5,미국 주요기업 주가등락률(마감)
6,미국 주요 ETF 등락률
7,미국시장 테마시황
8,[뉴욕증시] 테슬라·메타 급등에 일제히 상승…나스닥 2.69% ↑
9,국내외 주요 마감지수


In [ ]:
def naver_news_title(date_list):
  # 네이버 > 금융 > 많이본뉴스
  naver_url = 'https://finance.naver.com/news/news_list.naver?mode=RANK&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_result = []
  naver_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.simpleNewsList li')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_result.append([news_title])
          except:
            naver_error_cnt += 1
  return naver_result

In [ ]:
naver_news_rs = naver_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_news_rs length == ", len( naver_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=RANK&page=1&date=20220603
naver_news_rs length ==  25


In [ ]:
naver_title_df = pd.DataFrame(naver_news_rs, columns=['뉴스제목'])
#naver_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_title_df

,뉴스제목
0,"델리오, 첫 콜라보 NFT ‘클레이다이노 블루베이 에디션’ 경매.."
1,"[특징주] 한국조선해양, 수주 계약 취소에도… 주가는 2% 상승"
2,"거래소, ITX-AI 상장적격성 실질심사 사유 추가"
3,[특징주] 인천공항 국제선 정상화에 항공주 강세
4,[특징주] 우크라 외교차관 방한에 재건주 '강세'… 서암기계공업..
5,"[특징주] 청담글로벌, 코스닥 상장 첫날 공모가 대비 27%↑(.."
6,"금융위, 중기 특화 증권사 7곳 지정…케이프·DS 포함"
7,"에스티아이, 잉크젯 장비 수주 계약 체결"
8,"[특징주]디와이파워, 재건 중장비 '근육' 담당…우크라이나 재건.."
9,"상승장서 소외된 건설주, 선거 뒤 탄력받을까…증권가는 ""사라"""


In [ ]:
def naver_market_news_title(date_list):
  # 네이버 > 금융 > 뉴스포커스(시황/전망)
  naver_market_url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_market_result = []
  naver_market_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_market_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('dd.articleSubject')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_market_result.append([news_title])
          except:
            naver_market_error_cnt += 1
  return naver_market_result

In [ ]:
naver_market_news_rs = naver_market_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_market_news_rs length == ", len( naver_market_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page=1&date=20220603
naver_market_news_rs length ==  18


In [ ]:
naver_market_title_df = pd.DataFrame(naver_market_news_rs, columns=['뉴스제목'])
#naver_market_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_market_title_df

,뉴스제목
0,"한국투자증권, 미국 부동산 투자자문 서비스 개시"
1,[표]유가증권·코스닥 투자주체별 매매동향(5월 31일-최종치)
2,'목표가 93만원→48만원'…삼성SDI 혹평한 씨티증권에 '소나기 반박'
3,[표] 주간 수익률 베스트&워스트
4,[표] 주요 증시 지표 (6월 3일)
5,[ETF줌인] 명품 투자 ETF…면세점 매출 회복 수혜 기대
6,"대신증권, '비상장주식 안심거래 서비스' 실시"
7,[장외주식] 쓰리디팩토리 나흘 연속 상승
8,[표]코스피200지수 옵션 시세(6월 3일)
9,[표]코스피200지수·국채·달러 선물 시세(6월 3일)


## 금융 분야 감정분석기 : snunlp/KR-FinBert-SC
- https://huggingface.co/snunlp/KR-FinBert-SC
- https://github.com/snunlp/KR-FinBert-SC

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.0 MB/s 
     |████████████████████████████████| 86 kB 2.6 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 6.6 MB 14.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification # pre-trained된 모델 로딩
import torch

import urllib.request # 데이터 파일 URL다운로드
from tqdm import tqdm #모델 예측시 계산 진행상태를 보여줌

#from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset # 데이터셋을 분할하여 사용

import gc #GPU 캐쉬 초기화

In [4]:
print(torch.cuda.is_available() ) # GPU가용성 확인
print(torch.cuda.current_device() ) #현재 선택한 장치의 인덱스 
print(torch.cuda.device_count() ) #사용할 수 있는 GPU 수
print(torch.cuda.get_device_name(0) ) #장치 이름

False


RuntimeError: ignored

In [4]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

In [12]:
class Dataset_finance_emoton_csv(Dataset):
  def __init__(self, file_name):
    # --- 전처리 1 start ------------------------
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_emoton_data.csv")
    self.dataset = pd.read_csv('finance_emoton_data.csv')
    #self.dataset = pd.read_csv(file_name, sep=',')

    # tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):

    text = self.dataset['kor_sentence'][idx]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [15]:
class Dataset_samsung_news(Dataset):
  def __init__(self, file_name):
    # --- 전처리 1 start ------------------------
    # 자료 출처 : 빅카인즈 https://www.bigkinds.or.kr/
    # 삼성전자 뉴스검색결과를 미리 다운로드 하여 나의 github에 올려놓은 것을 사용함
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20180101-20181231.xlsx", filename="samsung_news_2018.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20190101-20191231.xlsx", filename="samsung_news_2019.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20200101-20201231.xlsx", filename="samsung_news_2020.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20210101-20211231.xlsx", filename="samsung_news_2021.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20220101-20220607.xlsx", filename="samsung_news_2022.xlsx")
    
    #self.dataset = pd.read_csv(file_name, sep=',')
    self.dataset = pd.read_excel('samsung_news_2018.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] )
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2019.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2020.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2021.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2022.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)

    # tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):

    text = self.dataset['제목'][idx]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [16]:
#sample_dataset = Dataset_finance_emoton_csv("")
sample_dataset = Dataset_samsung_news("")
sample_dataset.dataset

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Downloading:   0%|          | 0.00/372 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/140k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

,일자,언론사,제목,본문
0,20181231,한국일보,"화웨이, 글로벌 판매량 힘입어 프리미엄 스마트폰 첫 10%대 점유율",중국 화웨이가 3분기 프리미엄 스마트폰 시장에서 처음으로 10% 이상 점유율을 기록...
1,20181231,한국일보,증시 주저앉으며... 주식 100대 부자 자산 25조 증발,2018년 국내 증시가 주저 앉으면서 상장 주식 100대 부자의 자산이 한해 동안 ...
2,20181231,한국경제,"5G 보급 본격화, 헬스케어도 高성장 클라우드 엔터 자율車 바이오 '유망'",도전 2019 - 주식 시장 \n \n성장세 뚜렷한 테마는 \n \n \n[ 오형주...
3,20181231,서울경제,여명-눈물 닦아주는 기해년을 기대하며,홍준석 증권부장 \n \n연기자가 울고 예능인도 운다. 주말 내내 이어진 눈물의 시...
4,20181231,서울경제,2019 증시 키워드 GOLD PIG,지난해 천당과 지옥을 오갔던 국내 주식시장이 기해년 새해를 맞아 힘찬 도약을 준비하...
...,...,...,...,...
70332,20220101,매일경제,"""반도체부터 리츠까지"" 증권사가 '이구동성' 추천한 주식은?",올해 증권시장에서 코스피와 코스닥이 박스권으로 한 해의 거래를 마감해 아쉬움을 남기...
70333,20220101,세계일보,주식부호 톱 5위에 ‘이재용 홍라희 이부진 이서현’ 올라,"지난해 국내 증시 시가총액이 2020년에 비해 283조원 가까이 늘어난 가운데, 주..."
70334,20220101,서울경제,연말 효과 무색 코스피 '1월 효과'로 다시 날까 [다음주 증시전망],12월의 마지막 주 ‘연말 랠리’를 바라봤던 투자자들의 심경이 무색하게도 코스피는 ...
70335,20220101,한국경제,"""올 한국증시, 미국보다 나을 수도"" 펀드매니저들 입 모았다 [다시 열자! 3000시대]","지겨운 ‘코로나19’라는 단어에서 조금이라도 벗어나길 바랐지만, 여전히 주식시장에 ..."


In [14]:
# 모델 생성 : snunlp/KR-FinBert-SC
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC",num_labels=3).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert",num_labels=3)
#model.cuda()

In [51]:
# 데이터 로딩
batchSize = 30 # GPU 메모리 부족 오류나면 수치를 줄일것

sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

        labels                                           sentence  \
0      neutral  According to Gran, the company has no plans to...   
1      neutral  Technopolis plans to develop in stages an area...   
2     negative  The international electronic industry company ...   
3     positive  With the new production plant the company woul...   
4     positive  According to the company's updated strategy fo...   
...        ...                                                ...   
4841  negative  LONDON MarketWatch -- Share prices ended lower...   
4842   neutral  Rinkuskiai's beer sales fell by 6.5 per cent t...   
4843  negative  Operating profit fell to EUR 35.4 mn from EUR ...   
4844  negative  Net sales of the Paper segment decreased to EU...   
4845  negative  Sales in Finland decreased by 10.5 % in Januar...   

                                           kor_sentence  pred  
0     Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...     0  
1     테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기

In [53]:
model.eval() #모델 평가모드로 변경

# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

idx_s = 0
idx_e = 0

#label_dict = {0: 'negative', 1: 'neutral', 2: 'positive'}
print(model.config.id2label)

#예측 결과 초기화
sample_result['pred_label'] = 0
sample_result['negative'] = 0
sample_result['neutral'] = 0
sample_result['positive'] = 0

#for input_ids_batch, attention_masks_batch,y_batch in tqdm(sample_loader):
for input_ids_batch, attention_masks_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1) #예측한 원소중 max값 1개만 반환
  predictions = torch.nn.functional.softmax(y_pred, dim=-1) # 예측한 원소의 값이 0과 1사이의 값으로 변환 (원소 합은 1)
  #print(predictions)
  
  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  rs_negative = predictions[:, 0].tolist()
  rs_neutral = predictions[:, 1].tolist()
  rs_positive = predictions[:, 2].tolist()

  print(rsList)
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  idx_s = idx_e - len(rsList)
  print("index==", idx_s, idx_e)

  sample_result['pred_label'][idx_s : idx_e] = rsList #배치사이즈 만큼 예측된 값 일괄 저장
  sample_result['negative'][idx_s : idx_e] = rs_negative
  sample_result['neutral'][idx_s : idx_e] = rs_neutral
  sample_result['positive'][idx_s : idx_e] = rs_positive

print(sample_result)

{0: 'negative', 1: 'neutral', 2: 'positive'}


  0%|          | 0/162 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

[1, 2, 0, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2]
index== 0 30


RuntimeError: ignored

In [48]:
print(sample_result)

        labels                                           sentence  \
0      neutral  According to Gran, the company has no plans to...   
1      neutral  Technopolis plans to develop in stages an area...   
2     negative  The international electronic industry company ...   
3     positive  With the new production plant the company woul...   
4     positive  According to the company's updated strategy fo...   
...        ...                                                ...   
4841  negative  LONDON MarketWatch -- Share prices ended lower...   
4842   neutral  Rinkuskiai's beer sales fell by 6.5 per cent t...   
4843  negative  Operating profit fell to EUR 35.4 mn from EUR ...   
4844  negative  Net sales of the Paper segment decreased to EU...   
4845  negative  Sales in Finland decreased by 10.5 % in Januar...   

                                           kor_sentence  pred  
0     Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...     1  
1     테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기

In [ ]:
# 예측 결과를 파일로 저장
#sample_csv = sample_result.to_csv('sample.csv')
sample_csv = sample_result.to_csv('sample_result.csv',sep=',',na_rep='NaN', columns=['labels','kor_sentence','pred'],index=False)
# 파일을 PC로 다운로드 하기
from google.colab import files
files.download('sample_result.csv')

In [ ]:
#------------------------------------------------------------------------------------------
#*************************************
#*************************************

In [6]:
# 감정분석기 : snunlp/KR-FinBert-SC

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
#tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC",num_labels=3).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert",num_labels=3)
#model.cuda()

In [7]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_emoton_data.csv")

('finance_emoton_data.csv', <http.client.HTTPMessage at 0x7f24ef76a090>)

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20180101-20181231.xlsx", filename="samsung_news_2018.xlsx")

('samsung_news_2018.xlsx', <http.client.HTTPMessage at 0x7f24ef76ad10>)

In [9]:
finance_emoton_data = pd.read_csv('finance_emoton_data.csv')
finance_emoton_data

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,neutral,Rinkuskiai's beer sales fell by 6.5 per cent t...,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,negative,Net sales of the Paper segment decreased to EU...,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [10]:
#naver_market_title_df
#naver_title_df
#paxnet_title_df
#df_list3 = list(naver_title_df["뉴스제목"])
df_list3 = [
"현대바이오, '폴리탁셀' 코로나19 치료 가능성에 19% 급등" #Positive
,"이수화학, 3분기 영업익 176억…전년比 80%↑" #Positive
,"GKL, 7년 만에 두 자릿수 매출성장 예상" #Positive
,"위지윅스튜디오, 콘텐츠 활약에 사상 첫 매출 1000억원 돌파" #Positive
,"삼성전자, 2년 만에 인도 스마트폰 시장 점유율 1위 '왕좌 탈환'" #Positive
,"영화관株 '코로나 빙하기' 언제 끝나나… CJ CGV 올 4000억 손실 날수도" #Negative
,"C쇼크에 멈춘 흑자비행…대한항공 1분기 영업적자 566억" #Negative
,"'1000억대 횡령·배임' 최신원 회장 구속… SK네트웍스 경영 공백 방지 최선" #Negative
,"부품 공급 차질에…기아차 광주공장 전면 가동 중단" #Negative
,"현대제철, 지난해 영업익 3,313억원···전년比 67.7% 감소" #Negative
]
#df_list.append(df_list2)

df_list = list(finance_emoton_data["kor_sentence"])

In [ ]:
df_list

In [ ]:
#tokenize text to be sent to model
inputs = tokenizer(df_list
                   , max_length=256
                   , padding = True
                   , truncation = True
                   , return_tensors='pt'
                   ,add_special_tokens=True
                   #,return_attention_mask=True
                   ,pad_to_max_length=True
                   #,padding='max_length'
).to(device)

#outputs = model(**inputs)
outputs = model(**inputs)

In [7]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
#predictions2 = torch.nn.functional.softmax(outputs.logits, dim=1)

In [8]:
predictions

tensor([[3.4415e-05, 9.5938e-05, 9.9987e-01],
        [1.9011e-02, 2.0313e-04, 9.8079e-01],
        [2.8504e-05, 1.2070e-04, 9.9985e-01],
        [4.0596e-05, 7.6798e-05, 9.9988e-01],
        [4.5128e-05, 7.4144e-05, 9.9988e-01],
        [9.9981e-01, 9.8151e-05, 8.8476e-05],
        [9.9981e-01, 9.3540e-05, 9.2973e-05],
        [9.9969e-01, 1.9824e-04, 1.1543e-04],
        [9.9979e-01, 1.4074e-04, 7.1481e-05],
        [9.9982e-01, 9.9545e-05, 7.8000e-05]], grad_fn=<SoftmaxBackward0>)

In [9]:
#label_dict = {0: 'negative', 1: 'neutral', 2: 'positive'}
print(model.config.id2label)

{0: 'negative', 1: 'neutral', 2: 'positive'}


In [13]:
negative = predictions[:, 0].tolist()
neutral = predictions[:, 1].tolist()
positive = predictions[:, 2].tolist()

table = {'Headline':df_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral
         ,"label_Positive" : [ 1 if cc > 0.5 else 0 for cc in positive ] # 분류 : 긍정 1, 이외 0
         }
      
df2 = pd.DataFrame(table, columns = table.keys() )
df2["Score"] = df2["Positive"] - df2["Negative"] # 점수 : 긍정 지표 산출

In [15]:
# 일자별로 뉴스를 묶을때 사용
# df2.groupby("일자 컬럼").mean()

In [16]:
df2

,Headline,Positive,Negative,Neutral,label_Positive,Score
0,"현대바이오, '폴리탁셀' 코로나19 치료 가능성에 19% 급등",0.999870,0.000034,0.000096,1,0.999835
1,"이수화학, 3분기 영업익 176억…전년比 80%↑",0.980786,0.019011,0.000203,1,0.961774
2,"GKL, 7년 만에 두 자릿수 매출성장 예상",0.999851,0.000029,0.000121,1,0.999822
3,"위지윅스튜디오, 콘텐츠 활약에 사상 첫 매출 1000억원 돌파",0.999883,0.000041,0.000077,1,0.999842
4,"삼성전자, 2년 만에 인도 스마트폰 시장 점유율 1위 '왕좌 탈환'",0.999881,0.000045,0.000074,1,0.999836
5,영화관株 '코로나 빙하기' 언제 끝나나… CJ CGV 올 4000억 손실 날수도,0.000088,0.999813,0.000098,0,-0.999725
6,C쇼크에 멈춘 흑자비행…대한항공 1분기 영업적자 566억,0.000093,0.999813,0.000094,0,-0.999721
7,'1000억대 횡령·배임' 최신원 회장 구속… SK네트웍스 경영 공백 방지 최선,0.000115,0.999686,0.000198,0,-0.999571
8,부품 공급 차질에…기아차 광주공장 전면 가동 중단,0.000071,0.999788,0.000141,0,-0.999716
9,"현대제철, 지난해 영업익 3,313억원···전년比 67.7% 감소",0.000078,0.999823,0.000100,0,-0.999745


In [ ]:
df2.groupby("label_Positive").mean()

,Positive,Negative,Neutral
label_Positive,,,
0,0.000089,0.999785,0.000126
1,0.996054,0.003832,0.000114
